Step 2: Import required libraries and load the dataset


In [1]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic, SVD
from surprise.model_selection import cross_validate

movies = pd.read_csv('data/movies.csv')
ratings = pd.read_csv('data/ratings.csv')